<a href="https://colab.research.google.com/github/abhisheksuran/Atari_DQN/blob/master/PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# sh -c 'pip install tensorflow tensorflow-metal tqdm gymnasium[classic-control] tensorflow_probability ipywidgets matplotlib pyvirtualdisplay'
import sys
sys.path.append('../Simulator')

import motoflex_gym

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tqdm import tqdm
import gymnasium as gym
import argparse
import numpy as np

print(tf.__version__)

tf.config.set_visible_devices([], 'GPU')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

2.15.0


https://github.com/marload/DeepRL-TensorFlow2/blob/master/PPO/PPO_Continuous.py

From example above, however, it does not train at the moment.

In [2]:
tf.keras.backend.set_floatx('float64')

class Actor:
    def __init__(self, state_dim, action_dim, action_bound, std_bound):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.action_bound = action_bound
        self.std_bound = std_bound
        self.model = self.create_model()
        self.opt = tf.keras.optimizers.Adam(args.actor_lr)

    def get_action(self, state):
        state = np.reshape(state, [1, self.state_dim])
        mu, std = self.model.predict(state, verbose=0)
        action = np.random.normal(mu[0], std[0], size=self.action_dim)
        action = np.clip(action, -self.action_bound, self.action_bound)
        log_policy = self.log_pdf(mu, std, action)

        return log_policy, action

    def log_pdf(self, mu, std, action):
        std = tf.clip_by_value(std, self.std_bound[0], self.std_bound[1])
        var = std ** 2
        log_policy_pdf = -0.5 * (action - mu) ** 2 / \
            var - 0.5 * tf.math.log(var * 2 * np.pi)
        return tf.reduce_sum(log_policy_pdf, 1, keepdims=True)

    def create_model(self):
        state_input = Input((self.state_dim,))
        dense_1 = Dense(32, activation='relu')(state_input)
        dense_2 = Dense(32, activation='relu')(dense_1)
        out_mu = Dense(self.action_dim, activation='tanh')(dense_2)
        mu_output = Lambda(lambda x: x * self.action_bound)(out_mu)
        std_output = Dense(self.action_dim, activation='softplus')(dense_2)
        return tf.keras.models.Model(state_input, [mu_output, std_output])

    def compute_loss(self, log_old_policy, log_new_policy, actions, gaes):
        ratio = tf.exp(log_new_policy - tf.stop_gradient(log_old_policy))
        gaes = tf.stop_gradient(gaes)
        clipped_ratio = tf.clip_by_value(
            ratio, 1.0-args.clip_ratio, 1.0+args.clip_ratio)
        surrogate = -tf.minimum(ratio * gaes, clipped_ratio * gaes)
        return tf.reduce_mean(surrogate)

    def train(self, log_old_policy, states, actions, gaes):
        with tf.GradientTape() as tape:
            mu, std = self.model(states, training=True)
            log_new_policy = self.log_pdf(mu, std, actions)
            loss = self.compute_loss(
                log_old_policy, log_new_policy, actions, gaes)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss


class Critic:
    def __init__(self, state_dim):
        self.state_dim = state_dim
        self.model = self.create_model()
        self.opt = tf.keras.optimizers.Adam(args.critic_lr)

    def create_model(self):
        return tf.keras.Sequential([
            Input((self.state_dim,)),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(1, activation='linear')
        ])

    def compute_loss(self, v_pred, td_targets):
        mse = tf.keras.losses.MeanSquaredError()
        return mse(td_targets, v_pred)

    def train(self, states, td_targets):
        with tf.GradientTape() as tape:
            v_pred = self.model(states, training=True)
            assert v_pred.shape == td_targets.shape
            loss = self.compute_loss(v_pred, tf.stop_gradient(td_targets))
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss
    

class Args:
    gamma = 0.9
    update_interval = 1024
    actor_lr = 0.001
    critic_lr = 0.001
    clip_ratio = 0.2
    lmbda = 0.95
    epochs = 10
args = Args()


def gae_target(rewards, v_values, next_v_value, done):
    n_step_targets = np.zeros_like(rewards)
    gae = np.zeros_like(rewards)
    gae_cumulative = 0
    forward_val = 0

    if not done:
        forward_val = next_v_value

    for k in reversed(range(0, len(rewards))):
        delta = rewards[k] + args.gamma * forward_val - v_values[k]
        gae_cumulative = args.gamma * args.lmbda * gae_cumulative + delta
        gae[k] = gae_cumulative
        forward_val = v_values[k]
        n_step_targets[k] = gae[k] + v_values[k]
    return gae, n_step_targets

def list_to_batch(list):
    batch = list[0]
    for elem in list[1:]:
        batch = np.append(batch, elem, axis=0)
    return batch

env_name = 'Pendulum-v1'
env = gym.wrappers.StepAPICompatibility(gym.make(env_name), output_truncation_bool=False)


state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high[0]
std_bound = [1e-2, 1.0]

actor_opt = tf.keras.optimizers.Adam(args.actor_lr)
critic_opt = tf.keras.optimizers.Adam(args.critic_lr)
actor = Actor(state_dim, action_dim, action_bound, std_bound)
critic = Critic(state_dim)



/Users/civic/dev/MoToFlex/.venv/lib/python3.11/site-packages/gymnasium/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in (old) done step API which returns one bool instead of two.
  deprecation(


In [3]:


max_episodes = 10000
for ep in range(max_episodes):
    state_batch = []
    action_batch = []
    reward_batch = []
    old_policy_batch = []

    episode_reward, done = 0, False

    state, _ = env.reset()

    while not done:
        env.render()
        log_old_policy, action = actor.get_action(state)

        next_state, reward, done, _ = env.step(action)

        state = np.reshape(state, [1, state_dim])
        action = np.reshape(action, [1, 1])
        next_state = np.reshape(next_state, [1, state_dim])
        reward = np.reshape(reward, [1, 1])
        log_old_policy = np.reshape(log_old_policy, [1, 1])

        state_batch.append(state)
        action_batch.append(action)
        reward_batch.append((reward+8)/8) # Why +8? Maybe sign has changed with new versions.
        old_policy_batch.append(log_old_policy)

        if len(state_batch) >= args.update_interval or done:
            states = list_to_batch(state_batch)
            actions = list_to_batch(action_batch)
            rewards = list_to_batch(reward_batch)
            old_policys = list_to_batch(old_policy_batch)

            v_values = critic.model.predict(states, verbose=0)
            next_v_value = critic.model.predict(next_state, verbose=0) # No batch like the others?

            gaes, td_targets = gae_target(
                rewards, v_values, next_v_value, done)

            for epoch in range(args.epochs):
                actor_loss = actor.train(
                    old_policys, states, actions, gaes)
                critic_loss = critic.train(states, td_targets)

            state_batch = []
            action_batch = []
            reward_batch = []
            old_policy_batch = []

        episode_reward += reward[0][0]
        state = next_state[0]

    print('EP{} EpisodeReward={}'.format(ep, episode_reward))



/Users/civic/dev/MoToFlex/.venv/lib/python3.11/site-packages/gymnasium/envs/classic_control/pendulum.py:173: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("Pendulum-v1", render_mode="rgb_array")
  gym.logger.warn(


EP0 EpisodeReward=-1562.6533129608545
EP1 EpisodeReward=-1139.509778651463
EP2 EpisodeReward=-1566.8020709618925
EP3 EpisodeReward=-1168.0199239634228
EP4 EpisodeReward=-1618.6212655111715
EP5 EpisodeReward=-1049.8182734672828
EP6 EpisodeReward=-1463.216106662559
EP7 EpisodeReward=-1516.6114004834353
EP8 EpisodeReward=-1584.7969123308096
EP9 EpisodeReward=-1378.7188170646084
EP10 EpisodeReward=-1473.9960963525882
EP11 EpisodeReward=-1592.4272593928076
EP12 EpisodeReward=-1557.7795137251974
EP13 EpisodeReward=-1176.4335606145364
EP14 EpisodeReward=-1608.9070058911414
EP15 EpisodeReward=-1290.0069241038827
EP16 EpisodeReward=-1715.7167896754243
EP17 EpisodeReward=-1566.3592117113672
EP18 EpisodeReward=-1675.1771293390054
EP19 EpisodeReward=-1534.751594399259
EP20 EpisodeReward=-1253.2697817699902
EP21 EpisodeReward=-1169.335416307532
EP22 EpisodeReward=-1195.8207226330783
EP23 EpisodeReward=-933.5179947493388
EP24 EpisodeReward=-1458.8507598800263
EP25 EpisodeReward=-1431.4488715250545
E

KeyboardInterrupt: 